<center><img src="https://keras.io/img/logo-small.png" alt="Keras logo" width="100"><br/>
This starter notebook is provided by the Keras team.</center>

# HMS - Harmful Brain Activity Classification with [KerasCV](https://github.com/keras-team/keras-cv) and [Keras](https://github.com/keras-team/keras)

> The objective of this competition is to classify seizures and other patterns of harmful brain activity in critically ill patients

This notebook guides you through the process of training and inferring a Deep Learning model, specifically EfficientNetV2, using KerasCV on the competition dataset. Specificaclly, this notebook uses spectrogram of the eeg data to classify the patterns.

Fun fact: This notebook is backend-agnostic, supporting TensorFlow, PyTorch, and JAX. Utilizing KerasCV and Keras allows us to choose our preferred backend. Explore more details on [Keras](https://keras.io/keras_core/announcement/).

In this notebook, you will learn:

* Loading the data efficiently using [`tf.data`](https://www.tensorflow.org/guide/data).
* Creating the model using KerasCV presets.
* Training the model.
* Inference and Submission on test data.

**Note**: For a more in-depth understanding of KerasCV, refer to the [KerasCV guides](https://keras.io/guides/keras_cv/).

# 🛠 | Install Libraries  

Since internet access is **disabled** during inference, we cannot install libraries in the usual `!pip install <lib_name>` manner. Instead, we need to install libraries from local files. In the following cell, we will install libraries from our local files. The installation code stays very similar - we just use the `filepath` instead of the `filename` of the library. So now the code is `!pip install <local_filepath>`. 

> The `filepath` of these local libraries look quite complicated, but don't be intimidated! Also `--no-deps` argument ensures that we are not installing any additional libraries.

In [1]:
!pip install -q /kaggle/input/kerasv3-lib-ds/keras_cv-0.8.2-py3-none-any.whl --no-deps
!pip install -q /kaggle/input/kerasv3-lib-ds/tensorflow-2.15.0.post1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl --no-deps
!pip install -q /kaggle/input/kerasv3-lib-ds/keras-3.0.4-py3-none-any.whl --no-deps

# 📚 | Import Libraries 

In [2]:
import os
os.environ["KERAS_BACKEND"] = "jax" # you can also use tensorflow or torch

import keras_cv
import keras
from keras import ops
import tensorflow as tf

import cv2
import pandas as pd
import numpy as np
from glob import glob
from tqdm.notebook import tqdm
import joblib

import matplotlib.pyplot as plt 

2024-02-27 17:29:21.815571: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-27 17:29:21.815624: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-27 17:29:21.817123: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Library Versions

In [3]:
print("TensorFlow:", tf.__version__)
print("Keras:", keras.__version__)
print("KerasCV:", keras_cv.__version__)

TensorFlow: 2.15.0
Keras: 3.0.4
KerasCV: 0.8.2


# ⚙️ | Configuration

In [4]:
class CFG:
    verbose = 1  # Verbosity
    seed = 42  # Random seed
    preset = "efficientnetv2_b2_imagenet"  # Name of pretrained classifier
    image_size = [400, 300]  # Input image size
    epochs = 13 # Training epochs
    batch_size = 64  # Batch size
    lr_mode = "cos" # LR scheduler mode from one of "cos", "step", "exp"
    drop_remainder = True  # Drop incomplete batches
    num_classes = 6 # Number of classes in the dataset
    fold = 0 # Which fold to set as validation data
    class_names = ['Seizure', 'LPD', 'GPD', 'LRDA','GRDA', 'Other']
    label2name = dict(enumerate(class_names))
    name2label = {v:k for k, v in label2name.items()}

# ♻️ | Reproducibility 
Sets value for random seed to produce similar result in each run.

In [5]:
keras.utils.set_random_seed(CFG.seed)

# 📁 | Dataset Path 

In [6]:
BASE_PATH = "/kaggle/input/hms-harmful-brain-activity-classification"

SPEC_DIR = "/tmp/dataset/hms-hbac"
os.makedirs(SPEC_DIR+'/train_spectrograms', exist_ok=True)
os.makedirs(SPEC_DIR+'/test_spectrograms', exist_ok=True)

# 📖 | Meta Data 

In [7]:
# df = pd.read_csv(f'{BASE_PATH}/train.csv')
# df['total_evaluators'] = df[['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']].sum(axis=1)
# df['consensus'] = df[['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']].max(axis=1)
# df['consensus_column'] = df[['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']].idxmax(axis=1)
# df['row_agreement'] = df['consensus']/df['total_evaluators']
# original_df = df.copy(deep=True)
# large = original_df[original_df['total_evaluators'] >= 10].copy()
# small = original_df[original_df['total_evaluators'] < 10].copy()


# # Trying with using only the rows that have more than 5 evaluators in total
# df = original_df[original_df['total_evaluators'] >= 5].copy()
# df.head(25)

# Trying with using only the rows that have more than 0.7 agreement
# df = original_df[original_df['row_agreement'] >= 0.7].copy()
# df.head(25)

# row_agreement_agg = df.groupby('eeg_id')['consensus'].agg('sum')

# # Mapping this aggregated value back to each row in 'df'
# df['row_consensus_agg'] = df['eeg_id'].map(row_agreement_agg)

# row_evaluators_agg = df.groupby('eeg_id')['total_evaluators'].agg('sum')

# # Mapping this aggregated value back to each row in 'df'
# df['row_evaluators_agg'] = df['eeg_id'].map(row_evaluators_agg)

# df['eeg_agreement'] = df['row_consensus_agg']/df['row_evaluators_agg']

# original_df.head(25)  # Display the first few rows of the DataFrame to show the new column



In [8]:
# # Train + Valid small

# small['eeg_path'] = f'{BASE_PATH}/train_eegs/'+small['eeg_id'].astype(str)+'.parquet'
# small['spec_path'] = f'{BASE_PATH}/train_spectrograms/'+small['spectrogram_id'].astype(str)+'.parquet'
# small['spec2_path'] = f'{SPEC_DIR}/train_spectrograms/'+small['spectrogram_id'].astype(str)+'.npy'
# small['class_name'] = small.expert_consensus.copy()
# small['class_label'] = small.expert_consensus.map(CFG.name2label)
# display(small.head(2))

# # Train + Valid large

# large['eeg_path'] = f'{BASE_PATH}/train_eegs/'+large['eeg_id'].astype(str)+'.parquet'
# large['spec_path'] = f'{BASE_PATH}/train_spectrograms/'+large['spectrogram_id'].astype(str)+'.parquet'
# large['spec2_path'] = f'{SPEC_DIR}/train_spectrograms/'+large['spectrogram_id'].astype(str)+'.npy'
# large['class_name'] = large.expert_consensus.copy()
# large['class_label'] = large.expert_consensus.map(CFG.name2label)
# display(large.head(2))

# # Test
# test_df = pd.read_csv(f'{BASE_PATH}/test.csv')
# test_df['eeg_path'] = f'{BASE_PATH}/test_eegs/'+test_df['eeg_id'].astype(str)+'.parquet'
# test_df['spec_path'] = f'{BASE_PATH}/test_spectrograms/'+test_df['spectrogram_id'].astype(str)+'.parquet'
# test_df['spec2_path'] = f'{SPEC_DIR}/test_spectrograms/'+test_df['spectrogram_id'].astype(str)+'.npy'
# display(test_df.head(2))

## Convert `.parquet` to `.npy`

To facilitate easier data loading, we will convert the EEG spectrograms from `parquet` to `npy` format. This process involves saving the spectrogram data, and since the content of the files remains the same, no significant changes are made. 

> It's worth noting that the `time` column is excluded, as it is not part of the spectrogram.

In [9]:
# # Define a function to process a single eeg_id
# def process_spec(spec_id, split="train"):
#     spec_path = f"{BASE_PATH}/{split}_spectrograms/{spec_id}.parquet"
#     spec = pd.read_parquet(spec_path)
#     spec = spec.fillna(0).values[:, 1:].T # fill NaN values with 0, transpose for (Time, Freq) -> (Freq, Time)
#     spec = spec.astype("float32")
#     np.save(f"{SPEC_DIR}/{split}_spectrograms/{spec_id}.npy", spec)

# # Get unique spec_ids of train and valid data
# spec_ids_small = small["spectrogram_id"].unique()

# # Parallelize the processing using joblib for training data
# _ = joblib.Parallel(n_jobs=-1, backend="loky")(
#     joblib.delayed(process_spec)(spec_id, "train")
#     for spec_id in tqdm(spec_ids_small, total=len(spec_ids_small))
# )

# spec_ids_large = large["spectrogram_id"].unique()

# # Parallelize the processing using joblib for training data
# _ = joblib.Parallel(n_jobs=-1, backend="loky")(
#     joblib.delayed(process_spec)(spec_id, "train")
#     for spec_id in tqdm(spec_ids_large, total=len(spec_ids_large))
# )

# # Get unique spec_ids of test data
# test_spec_ids = test_df["spectrogram_id"].unique()

# # Parallelize the processing using joblib for test data
# _ = joblib.Parallel(n_jobs=-1, backend="loky")(
#     joblib.delayed(process_spec)(spec_id, "test")
#     for spec_id in tqdm(test_spec_ids, total=len(test_spec_ids))
# )

# 🍚 | DataLoader

This DataLoader first reads `npy` spectrogram files and extracts labeled subsamples using specified `offset` values. Then, it converts the spectrogram data into `log spectrogram` and applies the popular signal augmentation `MixUp`.

> Note that, we are converting the mono channel signal to a 3-channel signal for using "ImageNet" weights of pretrained model.

In [10]:
def build_augmenter(dim=CFG.image_size):
    augmenters = [
        keras_cv.layers.MixUp(alpha=2.0),
        keras_cv.layers.RandomCutout(height_factor=(1.0, 1.0),
                                     width_factor=(0.06, 0.1)), # freq-masking
        keras_cv.layers.RandomCutout(height_factor=(0.06, 0.1),
                                     width_factor=(1.0, 1.0)), # time-masking
    ]
    
    def augment(img, label):
        data = {"images":img, "labels":label}
        for augmenter in augmenters:
            if tf.random.uniform([]) < 0.5:
                data = augmenter(data, training=True)
        return data["images"], data["labels"]
    
    return augment


def build_decoder(with_labels=True, target_size=CFG.image_size, dtype=32):
    def decode_signal(path, offset=None):
        # Read .npy files and process the signal
        file_bytes = tf.io.read_file(path)
        sig = tf.io.decode_raw(file_bytes, tf.float32)
        sig = sig[1024//dtype:]  # Remove header tag
        sig = tf.reshape(sig, [400, -1])
        
        # Extract labeled subsample from full spectrogram using "offset"
        if offset is not None: 
            offset = offset // 2  # Only odd values are given
            sig = sig[:, offset:offset+300]
            
            # Pad spectrogram to ensure the same input shape of [400, 300]
            pad_size = tf.math.maximum(0, 300 - tf.shape(sig)[1])
            sig = tf.pad(sig, [[0, 0], [0, pad_size]])
            sig = tf.reshape(sig, [400, 300])
        
        # Log spectrogram 
        sig = tf.clip_by_value(sig, tf.math.exp(-4.0), tf.math.exp(8.0)) # avoid 0 in log
        sig = tf.math.log(sig)
        
        # Normalize spectrogram
        sig -= tf.math.reduce_mean(sig)
        sig /= tf.math.reduce_std(sig) + 1e-6
        
        # Mono channel to 3 channels to use "ImageNet" weights
        sig = tf.tile(sig[..., None], [1, 1, 3])
        return sig
    
    def decode_label(label):
        label = tf.one_hot(label, CFG.num_classes)
        label = tf.cast(label, tf.float32)
        label = tf.reshape(label, [CFG.num_classes])
        return label
    
    def decode_with_labels(path, offset=None, label=None):
        sig = decode_signal(path, offset)
        label = decode_label(label)
        return (sig, label)
    
    return decode_with_labels if with_labels else decode_signal


def build_dataset(paths, offsets=None, labels=None, batch_size=32, cache=True,
                  decode_fn=None, augment_fn=None,
                  augment=False, repeat=True, shuffle=1024, 
                  cache_dir="", drop_remainder=False):
    if cache_dir != "" and cache is True:
        os.makedirs(cache_dir, exist_ok=True)
    
    if decode_fn is None:
        decode_fn = build_decoder(labels is not None)
    
    if augment_fn is None:
        augment_fn = build_augmenter()
    
    AUTO = tf.data.experimental.AUTOTUNE
    slices = (paths, offsets) if labels is None else (paths, offsets, labels)
    
    ds = tf.data.Dataset.from_tensor_slices(slices)
    ds = ds.map(decode_fn, num_parallel_calls=AUTO)
    ds = ds.cache(cache_dir) if cache else ds
    ds = ds.repeat() if repeat else ds
    if shuffle: 
        ds = ds.shuffle(shuffle, seed=CFG.seed)
        opt = tf.data.Options()
        opt.experimental_deterministic = False
        ds = ds.with_options(opt)
    ds = ds.batch(batch_size, drop_remainder=drop_remainder)
    ds = ds.map(augment_fn, num_parallel_calls=AUTO) if augment else ds
    ds = ds.prefetch(AUTO)
    return ds

# 🔪 | Data Split

In the following code snippet, the data is divided into `5` folds. Note that, the `groups` argument is used to prevent any overlap of patients between the training and validation sets, thus avoiding potential **data leakage** issues. Additionally, each split is stratified based on the `class_label`, ensuring a uniform distribution of class labels in each fold.

In [11]:
# from sklearn.model_selection import StratifiedGroupKFold

# # Split for small
# sgkf = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=CFG.seed)

# small["fold"] = -1
# small.reset_index(drop=True, inplace=True)
# for fold, (train_idx, valid_idx) in enumerate(
#     sgkf.split(small, y=small["class_label"], groups=small["patient_id"])
# ):
#     small.loc[valid_idx, "fold"] = fold
# small.groupby(["fold", "class_name"])[["eeg_id"]].count().T
# # 
# # SPlit for large
# sgkf = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=CFG.seed)

# large["fold"] = -1
# large.reset_index(drop=True, inplace=True)
# for fold, (train_idx, valid_idx) in enumerate(
#     sgkf.split(large, y=large["class_label"], groups=large["patient_id"])
# ):
#     large.loc[valid_idx, "fold"] = fold
# large.groupby(["fold", "class_name"])[["eeg_id"]].count().T

## Build Train & Valid Dataset

Only first sample for each `spectrogram_id` is used in order to keep the dataset size managable. Feel free to train on full data.

In [12]:
# # Sample from full data for small
# # sample_df_small = small.groupby("spectrogram_id").head(2).reset_index(drop=True)
# # train_df_small = sample_df_small[sample_df_small.fold != CFG.fold]
# # valid_df_small = sample_df_small[sample_df_small.fold == CFG.fold]
# # print(f"# Num Train small: {len(train_df_small)} | Num Valid small: {len(valid_df_small)}")

# # # Train
# # train_paths_small = train_df_small.spec2_path.values
# # train_offsets_small = train_df_small.spectrogram_label_offset_seconds.values.astype(int)
# # train_labels_small = train_df_small.class_label.values
# # train_ds_small = build_dataset(train_paths_small, train_offsets_small, train_labels_small, batch_size=CFG.batch_size,
# #                          repeat=True, shuffle=True, augment=True, cache=True)

# # # Valid
# # valid_paths_small = valid_df_small.spec2_path.values
# # valid_offsets_small = valid_df_small.spectrogram_label_offset_seconds.values.astype(int)
# # valid_labels_small = valid_df_small.class_label.values
# # valid_ds_small = build_dataset(valid_paths_small, valid_offsets_small, valid_labels_small, batch_size=CFG.batch_size,
# #                          repeat=False, shuffle=False, augment=False, cache=True)



# # # Sample from full data for large
# sample_df_large = large.groupby("spectrogram_id").head(2).reset_index(drop=True)
# train_df_large = sample_df_large[sample_df_large.fold != CFG.fold]
# valid_df_large = sample_df_large[sample_df_large.fold == CFG.fold]
# print(f"# Num Train large: {len(train_df_large)} | Num Valid small: {len(valid_df_large)}")

# # Train
# train_paths_large = train_df_large.spec2_path.values
# train_offsets_large = train_df_large.spectrogram_label_offset_seconds.values.astype(int)
# train_labels_large = train_df_large.class_label.values
# train_ds_large = build_dataset(train_paths_large, train_offsets_large, train_labels_large, batch_size=CFG.batch_size,
#                          repeat=True, shuffle=True, augment=True, cache=True)

# # Valid
# valid_paths_large = valid_df_large.spec2_path.values
# valid_offsets_large = valid_df_large.spectrogram_label_offset_seconds.values.astype(int)
# valid_labels_large = valid_df_large.class_label.values
# valid_ds_large = build_dataset(valid_paths_large, valid_offsets_large, valid_labels_large, batch_size=CFG.batch_size,
#                          repeat=False, shuffle=False, augment=False, cache=True)

## Dataset Check

Let's visualize some samples from the dataset.

In [13]:
# imgs, tars = next(iter(train_ds_small))

# num_imgs = 8
# plt.figure(figsize=(4*4, num_imgs//4*5))
# for i in range(num_imgs):
#     plt.subplot(num_imgs//4, 4, i + 1)
#     img = imgs[i].numpy()[...,0]  # Adjust as per your image data format
#     img -= img.min()
#     img /= img.max() + 1e-4
#     tar = CFG.label2name[np.argmax(tars[i].numpy())]
#     plt.imshow(img)
#     plt.title(f"Target: {tar}")
#     plt.axis('off')
    
# plt.tight_layout()
# plt.show()

# 🔍 | Loss & Metric

The evaluation metric in this competition is **KL Divergence**, defined as,

$$
D_{\text{KL}}(P \parallel Q) = \sum_{i} P(i) \log\left(\frac{P(i)}{Q(i)}\right)
$$

Where:
- $P$ is the true distribution.
- $Q$ is the predicted distribution.

Interestingly, as KL Divergence is differentiable, we can directly use it as our loss function. Thus, we don't need to use a third-party metric like **Accuracy** to evaluate our model. Therefore, `valid_loss` can stand alone as an indicator for our evaluation. In keras, we already have impelementation for KL Divergence loss so we only need to import it.

In [14]:
LOSS = keras.losses.KLDivergence()

# 🤖 | Modeling

This notebook uses the `EfficientNetV2 B2` from KerasCV's collection of pretrained models. To explore other models, simply modify the `preset` in the `CFG` (config). Check the [KerasCV website](https://keras.io/api/keras_cv/models/tasks/image_classifier/) for a list of available pretrained models.

In [15]:
# # Build Classifier
# model_small = keras_cv.models.ImageClassifier.from_preset(
#     CFG.preset, num_classes=CFG.num_classes
# )

# # Compile the model  
# model_small.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-4),
#               loss=LOSS)

# # Model Sumamry
# model_small.summary()

# ⚓ | LR Schedule

A well-structured learning rate schedule is essential for efficient model training, ensuring optimal convergence and avoiding issues such as overshooting or stagnation.

In [16]:
import math

def get_lr_callback(batch_size=8, mode='cos', epochs=10, plot=False):
    lr_start, lr_max, lr_min = 5e-5, 6e-6 * batch_size, 1e-5
    lr_ramp_ep, lr_sus_ep, lr_decay = 3, 0, 0.75

    def lrfn(epoch):  # Learning rate update function
        if epoch < lr_ramp_ep: lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start
        elif epoch < lr_ramp_ep + lr_sus_ep: lr = lr_max
        elif mode == 'exp': lr = (lr_max - lr_min) * lr_decay**(epoch - lr_ramp_ep - lr_sus_ep) + lr_min
        elif mode == 'step': lr = lr_max * lr_decay**((epoch - lr_ramp_ep - lr_sus_ep) // 2)
        elif mode == 'cos':
            decay_total_epochs, decay_epoch_index = epochs - lr_ramp_ep - lr_sus_ep + 3, epoch - lr_ramp_ep - lr_sus_ep
            phase = math.pi * decay_epoch_index / decay_total_epochs
            lr = (lr_max - lr_min) * 0.5 * (1 + math.cos(phase)) + lr_min
        return lr

    if plot:  # Plot lr curve if plot is True
        plt.figure(figsize=(10, 5))
        plt.plot(np.arange(epochs), [lrfn(epoch) for epoch in np.arange(epochs)], marker='o')
        plt.xlabel('epoch'); plt.ylabel('lr')
        plt.title('LR Scheduler')
        plt.show()

    return keras.callbacks.LearningRateScheduler(lrfn, verbose=False)  # Create lr callback

In [17]:
# lr_cb = get_lr_callback(CFG.batch_size, mode=CFG.lr_mode, plot=True)

# 💾 | Model Checkpointing

In [18]:
# ckpt_cb = keras.callbacks.ModelCheckpoint("best_model_small.keras",
#                                          monitor='val_loss',
#                                          save_best_only=True,
#                                          save_weights_only=False,
#                                          mode='min')

# 🚂 | Training

In [19]:
# history = model_small.fit(
#     train_ds_small, 
#     epochs=CFG.epochs,
#     callbacks=[lr_cb, ckpt_cb], 
#     steps_per_epoch=len(train_df_small)//CFG.batch_size,
#     validation_data=valid_ds_small, 
#     verbose=CFG.verbose
# )

## Get preds for small



In [20]:
# model_small.load_weights("best_model_small.keras")

In [21]:
# test_paths = test_df.spec2_path.values
# test_ds = build_dataset(test_paths, batch_size=min(CFG.batch_size, len(test_df)),
#                          repeat=False, shuffle=False, cache=False, augment=False)

In [22]:
# preds_small = model_small.predict(test_ds)

In [23]:
# preds_small

In [24]:
# import gc
# from keras import backend as k
# # Free up memory
# k.clear_session()
# del model_small
# gc.collect()

# Training for large

In [25]:
# # Build Classifier
# model_large = keras_cv.models.ImageClassifier.from_preset(
#     CFG.preset, num_classes=CFG.num_classes
# )

# # Compile the model  
# model_large.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-4),
#               loss=LOSS)

# # Model Sumamry
# model_large.summary()

# lr_cb = get_lr_callback(CFG.batch_size, mode=CFG.lr_mode, plot=True)

# ckpt_cb = keras.callbacks.ModelCheckpoint("best_model_large.keras",
#                                          monitor='val_loss',
#                                          save_best_only=True,
#                                          save_weights_only=False,
#                                          mode='min')

In [26]:
# history = model_large.fit(
#     train_ds_large, 
#     epochs=CFG.epochs,
#     callbacks=[lr_cb, ckpt_cb], 
#     steps_per_epoch=len(train_df_large)//CFG.batch_size,
#     validation_data=valid_ds_large, 
#     verbose=CFG.verbose
# )

# 🧪 | Prediction

## Load Best Model

In [27]:
# model_large.load_weights("best_model_large.keras")

## Build Test Dataset

## Inference

In [28]:
# preds_large = model_large.predict(test_ds)

In [29]:
# k.clear_session()
# del model_large
# gc.collect()

# Load models and predict

In [30]:
test_df = pd.read_csv(f'{BASE_PATH}/test.csv')
test_df['eeg_path'] = f'{BASE_PATH}/test_eegs/'+test_df['eeg_id'].astype(str)+'.parquet'
test_df['spec_path'] = f'{BASE_PATH}/test_spectrograms/'+test_df['spectrogram_id'].astype(str)+'.parquet'
test_df['spec2_path'] = f'{SPEC_DIR}/test_spectrograms/'+test_df['spectrogram_id'].astype(str)+'.npy'
display(test_df.head(2))

def process_spec(spec_id, split="train"):
    spec_path = f"{BASE_PATH}/{split}_spectrograms/{spec_id}.parquet"
    spec = pd.read_parquet(spec_path)
    spec = spec.fillna(0).values[:, 1:].T # fill NaN values with 0, transpose for (Time, Freq) -> (Freq, Time)
    spec = spec.astype("float32")
    np.save(f"{SPEC_DIR}/{split}_spectrograms/{spec_id}.npy", spec)
    
    
test_spec_ids = test_df["spectrogram_id"].unique()

# Parallelize the processing using joblib for test data
_ = joblib.Parallel(n_jobs=-1, backend="loky")(
    joblib.delayed(process_spec)(spec_id, "test")
    for spec_id in tqdm(test_spec_ids, total=len(test_spec_ids))
)

test_paths = test_df.spec2_path.values
test_ds = build_dataset(test_paths, batch_size=min(CFG.batch_size, len(test_df)),
                         repeat=False, shuffle=False, cache=False, augment=False)

,spectrogram_id,eeg_id,patient_id,eeg_path,spec_path,spec2_path
0,853520,3911565283,6885,/kaggle/input/hms-harmful-brain-activity-class...,/kaggle/input/hms-harmful-brain-activity-class...,/tmp/dataset/hms-hbac/test_spectrograms/853520...


  0%|          | 0/1 [00:00<?, ?it/s]

In [31]:
!cp -r "/kaggle/input/large-votes/keras/version1/1/best_model_large.keras" ./

In [32]:
!cp -r "/kaggle/input/small-votes/keras/version1/1/best_model_small-2.keras" ./

In [33]:
model_large = keras.models.load_model("/kaggle/working/best_model_large.keras")

In [34]:
model_small = keras.models.load_model("/kaggle/working/best_model_small-2.keras")

In [35]:
preds_large = model_large.predict(test_ds)
preds_small = model_small.predict(test_ds)

1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step


# 📩 | Submission

In [36]:
preds = 0.6 * preds_large + 0.4 * preds_small

In [37]:
pred_df = test_df[["eeg_id"]].copy()
target_cols = [x.lower()+'_vote' for x in CFG.class_names]
pred_df[target_cols] = preds.tolist()

sub_df = pd.read_csv(f'{BASE_PATH}/sample_submission.csv')
sub_df = sub_df[["eeg_id"]].copy()
sub_df = sub_df.merge(pred_df, on="eeg_id", how="left")
sub_df.to_csv("submission.csv", index=False)
sub_df.head()

,eeg_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,3911565283,0.07882,0.422024,0.006418,0.405592,0.021944,0.065202


# 📌 | Reference
* [HMS-HBAC: ResNet34d Baseline [Training]](https://www.kaggle.com/code/ttahara/hms-hbac-resnet34d-baseline-training) 
* [EfficientNetB2 Starter - [LB 0.57]](https://www.kaggle.com/code/cdeotte/efficientnetb2-starter-lb-0-57)